In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp evaluation

# Evaluation
> Model performance evaluation

In [ ]:
#| export
from typing import Callable, List, Optional

import numpy as np
import pandas as pd

from utilsforecast.compat import DataFrame, pl_DataFrame, pl_concat

In [ ]:
#| export
def evaluate(
    y_hat_df: DataFrame,
    metrics: List[Callable],
    models: Optional[List[str]] = None,
    _y_train_df: Optional[DataFrame] = None,
    id_col: str = 'unique_id',
    time_col: str = 'ds',
    target_col: str = 'y',
) -> DataFrame:
    """Evaluate forecast using different metrics.
    
    Parameters
    ----------
    y_hat_df : pandas or polars DataFrame
        Forecasts to evaluate.
        Must have `id_col`, `time_col`, `target_col` and models' predictions.
    metrics : list of callable
        Functions with arguments `y`, `y_hat`, and optionally `y_train`.
    models : list of str, optional (default=None)
        Names of the models to evaluate.
        If `None` will use every column in the dataframe removing id, time and target.
    y_train_df : pandas DataFrame, optional (default=None)
        Training set. Used to evaluate metrics such as `mase`. 
    id_col : str (default='unique_id')
        Column that identifies each serie. If 'index' then the index is used.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars DataFrame
        Metrics with one row per (id, metric) combination and one column per model.
    """
    is_pandas = isinstance(y_hat_df, pd.DataFrame)
    if is_pandas:
        assert isinstance(y_hat_df, pd.DataFrame)  # mypy
        y_hat_df = y_hat_df.sort_values([id_col, time_col])
    else:
        y_hat_df = y_hat_df.sort([id_col, time_col])
    if models is None:
        model_cols = [c for c in y_hat_df.columns if c not in [id_col, time_col, target_col]]
    else:
        model_cols = models
    targets = y_hat_df[target_col].to_numpy()
    predictions = y_hat_df[model_cols].to_numpy().T
    sizes = y_hat_df[id_col].value_counts()
    if is_pandas:
        sizes = sizes.rename('counts').sort_index().reset_index()
    else:
        sizes = sizes.sort(id_col)
    indptr = np.append(np.uint32(0), sizes['counts'].to_numpy().cumsum())
    n_series = len(indptr) - 1
    results_per_metric = []
    for metric in metrics:
        results = {id_col: sizes[id_col], 'metric': metric.__name__}
        for i, model in enumerate(model_cols):
            model_results = np.empty(n_series)
            for j in range(n_series):
                sl = slice(indptr[j], indptr[j + 1])
                model_results[j] = metric(targets[sl], predictions[i, sl])
            results[model] = model_results
        df = pd.DataFrame(results) if is_pandas else pl_DataFrame(results)
        results_per_metric.append(df)
    if is_pandas:
        df = pd.concat(results_per_metric)
    else:
        df = pl_concat(results_per_metric)
    return df

In [ ]:
#| hide
from nbdev import show_doc

In [ ]:
show_doc(evaluate)

---

### evaluate

>      evaluate (y_hat_df:Union[pandas.core.frame.DataFrame,polars.dataframe.fra
>                me.DataFrame], metrics:List[Callable],
>                models:Optional[List[str]]=None, y_train_df:Union[pandas.core.f
>                rame.DataFrame,polars.dataframe.frame.DataFrame,NoneType]=None,
>                id_col:str='unique_id', time_col:str='ds', target_col:str='y')

Evaluate forecast using different metrics.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y_hat_df | Union |  | Forecasts to evaluate.<br>Must have `id_col`, `time_col`, `target_col` and models' predictions. |
| metrics | List |  | Functions with arguments `y`, `y_hat`, and optionally `y_train`. |
| models | Optional | None | Names of the models to evaluate.<br>If `None` will use every column in the dataframe removing id, time and target. |
| y_train_df | Union | None | Training set. Used to evaluate metrics such as `mase`.  |
| id_col | str | unique_id | Column that identifies each serie. If 'index' then the index is used. |
| time_col | str | ds | Column that identifies each timestep, its values can be timestamps or integers. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **Union** |  | **Metrics with one row per (id, metric) combination and one column per model.** |

In [ ]:
import pandas as pd
import polars as pl

from utilsforecast.losses import *

In [ ]:
subm = pd.read_csv('submission.csv')
subm = subm.melt(id_vars=['id'])
subm['model1'] = subm['value'] * 0.1 * np.random.rand(subm.shape[0])
subm['model2'] = subm['value'] * 0.1 * np.random.rand(subm.shape[0])
subm.head()

In [ ]:
%time evaluate(subm, metrics=[mape, smape, rmse, mae], id_col='id', time_col='variable', target_col='value').sort_values(['metric', 'id']).head()

CPU times: user 13.7 s, sys: 129 ms, total: 13.8 s
Wall time: 13.6 s


,id,metric,model1,model2
0,FOODS_1_001_CA_1_evaluation,mae,0.693528,0.686560
1,FOODS_1_001_CA_1_validation,mae,0.000000,0.000000
2,FOODS_1_001_CA_2_evaluation,mae,0.829257,0.815626
3,FOODS_1_001_CA_2_validation,mae,0.000000,0.000000
4,FOODS_1_001_CA_3_evaluation,mae,0.934637,0.934619


In [ ]:
pl_df = pl.from_pandas(subm)

In [ ]:
%time evaluate(pl_df, metrics=[mape, smape, rmse, mae], id_col='id', time_col='variable', target_col='value').sort(['metric', 'id']).head()

CPU times: user 14 s, sys: 262 ms, total: 14.3 s
Wall time: 13 s


id,metric,model1,model2
str,str,f64,f64
"""FOODS_1_001_CA…","""mae""",0.693528,0.68656
"""FOODS_1_001_CA…","""mae""",0.0,0.0
"""FOODS_1_001_CA…","""mae""",0.829257,0.815626
"""FOODS_1_001_CA…","""mae""",0.0,0.0
"""FOODS_1_001_CA…","""mae""",0.934637,0.934619
